### Importing libraries


In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 
import os
from keras.utils import to_categorical
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np



### Loading the dataset

In [2]:

# Paths
train_path = r'C:\Personal\AD LAB\Cats&Dogs\training_set'
test_path = r'C:\Personal\AD LAB\Cats&Dogs\test_set'

# Function to load images and labels
def load_data(data_path):
    images, labels = [], []
    for label, folder_name in enumerate(['cats', 'dogs']):  # 0 for cats, 1 for dogs
        folder = os.path.join(data_path, folder_name)
        for file_name in os.listdir(folder):
            file_path = os.path.join(folder, file_name)
            image = cv2.imread(file_path)
            if image is not None:
                images.append(cv2.resize(image, (100, 100)))
                labels.append(label)
    return np.array(images), np.array(labels)

# Load training and test data
X_train, y_train = load_data(train_path)
X_test, y_test = load_data(test_path)

# Print the shapes for verification
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (8005, 100, 100, 3), y_train shape: (8005,)
X_test shape: (2023, 100, 100, 3), y_test shape: (2023,)


In [3]:
# Flatten the image data
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

In [4]:
# Normalize the data for better performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)


In [5]:
# Initialize and train K-Means
kmeans = KMeans(n_clusters=2, random_state=42)  # 2 clusters for binary classification
kmeans.fit(X_train_scaled)

KMeans(n_clusters=2, random_state=42)

In [6]:
# Predict cluster labels for training and test sets
train_cluster_labels = kmeans.predict(X_train_scaled)
test_cluster_labels = kmeans.predict(X_test_scaled)

In [7]:
# Map cluster labels to match true labels
# (K-Means assigns arbitrary cluster labels, so we need to map them to the actual classes)
from scipy.stats import mode

In [8]:
# Convert one-hot encoded labels to binary
y_train_binary = np.argmax(y_train, axis=1) if len(y_train.shape) > 1 else y_train
y_test_binary = np.argmax(y_test, axis=1) if len(y_test.shape) > 1 else y_test


In [10]:
train_label_mapping = {}
for cluster in np.unique(train_cluster_labels):
    actual_label = mode(y_train[train_cluster_labels == cluster])[0][0]
    train_label_mapping[cluster] = actual_label

IndexError: invalid index to scalar variable.

In [20]:
# Map predicted clusters to actual labels
mapped_train_labels = np.array([train_label_mapping[label] for label in train_cluster_labels])
mapped_test_labels = np.array([train_label_mapping[label] for label in test_cluster_labels])



In [23]:
# Calculate accuracy
train_accuracy = accuracy_score(y_train, mapped_train_labels)
test_accuracy = accuracy_score(y_test, mapped_test_labels)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets